In [3]:
from src.goal_interpretation.goal_interpretation import run_model
from src.task_generation.task_generation import generate_graph_and_task
from langchain_ollama import ChatOllama
import json
from pprint import pprint

In [8]:
from src.action_sequencing.prompt_specification import specificate_prompt
from warnings import filterwarnings
filterwarnings('ignore')

test_sample = specificate_prompt(2, 10)

Goal interpretation completed at iteration 1
Subgoal decomposition completed at iteration 1


In [10]:
llm = ChatOllama( 
    model="qwen3:8b",
    temperature=0.0,
    reasoning=False,
)

In [11]:
response = llm.invoke(test_sample[0])

In [14]:
response

AIMessage(content='=== domain.pddl ===\n(define (domain home-robot)\n  (:requirements :strips :typing)\n  (:types agent object)\n  (:predicates\n    (on ?o1 - object ?o2 - object)\n    (facing ?a - agent ?o - object)\n    (inside ?a - agent ?r - object)\n    (next_to ?a - agent ?o - object)\n    (clean ?o - object)\n    (closed ?o - object)\n    (off ?o - object)\n    (holds_lh ?a - agent ?o - object)\n    (holds_rh ?a - agent ?o - object)\n    (close ?o1 - object ?o2 - object)\n    (between ?o1 - object ?o2 - object ?o3 - object)\n  )\n  (:action walk\n    :parameters (?a - agent ?o - object)\n    :precondition ()\n    :effect (next_to ?a ?o)\n  )\n  (:action turnto\n    :parameters (?a - agent ?o - object)\n    :precondition (next_to ?a ?o)\n    :effect (facing ?a ?o)\n  )\n  (:action run\n    :parameters (?a - agent ?o - object)\n    :precondition (facing ?a ?o)\n    :effect (close ?a ?o)\n  )\n  (:action find\n    :parameters (?a - agent ?o - object)\n    :precondition ()\n    :eff

In [13]:
pprint(response.content)

('=== domain.pddl ===\n'
 '(define (domain home-robot)\n'
 '  (:requirements :strips :typing)\n'
 '  (:types agent object)\n'
 '  (:predicates\n'
 '    (on ?o1 - object ?o2 - object)\n'
 '    (facing ?a - agent ?o - object)\n'
 '    (inside ?a - agent ?r - object)\n'
 '    (next_to ?a - agent ?o - object)\n'
 '    (clean ?o - object)\n'
 '    (closed ?o - object)\n'
 '    (off ?o - object)\n'
 '    (holds_lh ?a - agent ?o - object)\n'
 '    (holds_rh ?a - agent ?o - object)\n'
 '    (close ?o1 - object ?o2 - object)\n'
 '    (between ?o1 - object ?o2 - object ?o3 - object)\n'
 '  )\n'
 '  (:action walk\n'
 '    :parameters (?a - agent ?o - object)\n'
 '    :precondition ()\n'
 '    :effect (next_to ?a ?o)\n'
 '  )\n'
 '  (:action turnto\n'
 '    :parameters (?a - agent ?o - object)\n'
 '    :precondition (next_to ?a ?o)\n'
 '    :effect (facing ?a ?o)\n'
 '  )\n'
 '  (:action run\n'
 '    :parameters (?a - agent ?o - object)\n'
 '    :precondition (facing ?a ?o)\n'
 '    :effect (close

In [5]:
import json, re, subprocess
from pathlib import Path
from typing import TypedDict, Sequence, Annotated
from dotenv import load_dotenv
from langchain_core.messages import AIMessage, HumanMessage, BaseMessage, ToolMessage, SystemMessage
from langchain_ollama import ChatOllama
from langchain_core.tools import tool
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, START, END
from warnings import filterwarnings

from src.action_sequencing.raw_prompt import prompt
from src.task_generation.task_generation import generate_graph_and_task
from src.action_sequencing.prompt_specification import specificate_prompt

filterwarnings('ignore')
load_dotenv()

False

In [155]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    scene_graph: dict
    subgoal_list : list[str]
    

In [156]:
def validate_pddl_output(pddl_text) -> tuple[bool, str]:
    if "=== domain.pddl ===" not in pddl_text:
        return False, "Missing domain.pddl"
    if "=== problem.pddl ===" not in pddl_text:
        return False, "Missing problem.pddl"
    if pddl_text.count("=== domain.pddl ===") > 1:
        return False, "Duplicate domain.pddl"
    if pddl_text.count("=== problem.pddl ===") > 1:
        return False, "Duplicate problem.pddl"
    
    pddl_paths = (Path.cwd() / ".." / ".." / "ff-planner-docker" / "actual_plans").resolve()
    domain_path, problem_path = pddl_paths / "domain.pddl", pddl_paths / "problem.pddl"

    pattern = r"=== domain\.pddl ===\s*(.*?)\s*=== problem\.pddl ===\s*(.*)"
    match = re.search(pattern, pddl_text, re.DOTALL)
    
    domain_text, problem_text = match.group(1).strip(), match.group(2).strip()
    
    with open(domain_path, "w", encoding='utf-8') as f:
        f.write(domain_text)
    with open(problem_path, "w", encoding='utf-8') as f:
        f.write(problem_text)

    return True, "OK"

def run_planner(domain_name : str, problem_name : str) -> str:
    """
    Runs Fast Downward classic PDDL planner and returns the optimal plan if possible.
    Arguments: 
    domain_path - the name of generated domain file, e.g. "domain.pddl"
    problem_name - the name of generated problem file, e.g. "problem.pddl"
    Note that those names are the same as you generated before.
    If you've made many attempts and the plan is still unsolvable, please,
    write special code __plan_unsolvable__ to stop iterations.
    """
    base_path = (Path.cwd() / ".." / ".." / "ff-planner-docker" / "actual_plans").resolve()
    
    # Используем только имена файлов для передачи в контейнер
    domain_filename = Path(domain_name).name  # "domain.pddl"
    problem_filename = Path(problem_name).name  # "problem.pddl"

    # Путь к плану
    plan_filename = "plan.pddl"
    plan_file_host = base_path / plan_filename  # Путь на хосте
    plan_file_container = f"/planning/{plan_filename}" # Путь на докер образе 
    
    # Монтируем всю папку base_path в /planning
    cmd = [
        "docker", "run", "--rm",
        "--memory=1g",
        "-v", f"{base_path}:/planning",
        "downward-planner",
        "--plan-file", plan_file_container,
        f"/planning/{domain_filename}",
        f"/planning/{problem_filename}",
        "--search", "astar(lmcut())",
    ]
    # удаляем старый план во избежание багов
    if plan_file_host.exists():
        plan_file_host.unlink() 
        
    result = subprocess.run(cmd, capture_output=True, text=True, encoding='utf-8')
    
    if result.returncode  == 0:
        if plan_file_host.exists():
            with open(plan_file_host, "r", encoding="utf-8") as f:
                plan_content = f.read().strip()
            return plan_content
        
    log = result.stderr if result.stderr else result.stdout
    if 1 <= result.returncode  < 10:
        return "Partly successful termination: at least one plan was \
                        found and another component ran out of memory."
    elif 10 <= result.returncode  < 20:
        return "Unsuccessful, but error-free termination: task is unsolvable."
    elif 20 <= result.returncode  < 30:
        return "Expected failures which prevent the execution of further components: \
                                                                    OOM / Timeout."
    else:
        return f"Unrecoverable failure: {log}"

@tool
def plan_from_pddl(pddl_text: str) -> str:
    """
    Parses the pddl_text string and tries to run it using PDDL planner.
    Output is the plan if possible, otherwise error message is returned.
    
    The input pddl_text should contain a valid PDDL description of the task:
    === domain.pddl ===
    ...
    === problem.pddl ===
    ...
    """
    is_valid, message = validate_pddl_output(pddl_text)
    if not is_valid:
        return f"PDDL parsing error: {message}"
    
    plan_result = run_planner("domain.pddl", "problem.pddl")
    return plan_result
@tool
def find_object(object_name: str, graph: dict = None) -> str:
    """Finds an object on scene by it's name (with synonims) and returns it's id, states, properties."""

    base_folder = Path.cwd() / "../../virtualhome/resources/"
    base_folder.resolve()
    all_states_path = base_folder / "object_states.json"
    all_properties_path = base_folder / "properties_data.json"
    synonyms_path = base_folder / "class_name_equivalence.json"
    with open (all_states_path, "r", encoding = 'utf-8') as f:
        all_states = json.load(f)
    with open (all_properties_path, "r", encoding = 'utf-8') as f:
        all_properties = json.load(f)
    with open (synonyms_path, "r", encoding = 'utf-8') as f:
        synonyms = json.load(f)

    object = ""
    known_ids = {}
    for node in graph['nodes']:
        raw_name, obj_id = node['class_name'], node['id']
        candidate_names = [raw_name] + synonyms.get(raw_name, [])
        if object_name not in candidate_names:
            continue

        states = [s.upper() for s in node.get('states', [])]

        known_ids[obj_id] = raw_name
        possible_states, properties = [], []
        
        for name in candidate_names:
            if name in all_states:
                possible_states = [s.upper() for s in all_states[name]]
                break
        for name in candidate_names:
            if name in all_properties:
                properties = [s.upper() for s in all_properties[name]]
                break  

        object = f"{raw_name}, id: {obj_id}, states: {states}, possible states: {possible_states}, properties: {properties}\n"

    return object

@tool
def get_relations(object_id: int, graph: dict = None) -> str:
    """Return all edges, connected with this node by id (both directions)"""

    connections = []

    for edge in graph['edges']:
        if edge['from_id'] == object_id or edge['to_id'] == object_id:
            to_node = edge['to_id'] if edge['from_id'] == object_id else object_id
            from_node = edge['from_id'] if edge['to_id'] == object_id else object_id
            
            connection = f"{to_node} IS {edge['relation_type']} TO {from_node}"
            connections.append(connection)
    return "\n".join(connections) or [{"info": "No relations found."}]

In [157]:
tools = [plan_from_pddl, find_object, get_relations]
llm = ChatOllama(
    model="qwen3:8b",
    temperature=0.0,
    reasoning=False,
).bind_tools(tools)

In [159]:
def my_agent(state: AgentState):
                                
    all_messages = list(state["messages"]) 
    
    response = llm.invoke(all_messages)

    print(f"\n AI: {response.content}")
    if hasattr(response, "tool_calls") and response.tool_calls:
        print(f"USING TOOLS: {[tc['name'] for tc in response.tool_calls]}")

    return {"messages": [response]}

In [ ]:
_,it1=generate_graph_and_task(2)
it1['edges']

[{'from_id': 107, 'relation_type': 'FACING', 'to_id': 176}, {'from_id': 107, 'relation_type': 'FACING', 'to_id': 170}, {'from_id': 107, 'relation_type': 'FACING', 'to_id': 174}, {'from_id': 107, 'relation_type': 'FACING', 'to_id': 175}, {'from_id': 80, 'relation_type': 'INSIDE', 'to_id': 67}, {'from_id': 290, 'relation_type': 'INSIDE', 'to_id': 201}, {'from_id': 228, 'relation_type': 'ON', 'to_id': 205}, {'from_id': 116, 'relation_type': 'INSIDE', 'to_id': 67}, {'from_id': 116, 'relation_type': 'INSIDE', 'to_id': 108}, {'from_id': 216, 'relation_type': 'ON', 'to_id': 211}, {'from_id': 335, 'relation_type': 'FACING', 'to_id': 402}, {'from_id': 335, 'relation_type': 'FACING', 'to_id': 403}, {'from_id': 335, 'relation_type': 'FACING', 'to_id': 404}, {'from_id': 335, 'relation_type': 'FACING', 'to_id': 410}, {'from_id': 2005, 'relation_type': 'INSIDE', 'to_id': 319}, {'from_id': 64, 'relation_type': 'INSIDE', 'to_id': 1}, {'from_id': 361, 'relation_type': 'INSIDE', 'to_id': 358}, {'from_id

In [172]:
"status" in "{\"status\": \"fail\", \"message\": \"Plan is infeasible\"}" 

True

In [ ]:
def update_subgoals_from_scene(state: AgentState) -> None:
    """
    Updates state["subgoal_list"] by removing goals that are satisfied in state["scene_graph"].
    Assumes both are lists of strings (e.g., "NEXT_TO(robot.1, toilet.37)").
    """

    """ current_node_states = set(state["scene_graph"]['nodes']) 
    current_edge_states = set(state["scene_graph"]['edges']) 
    remaining_goals = []

    for goal in state["subgoal_list"]:
        if goal in current_predicates:
            continue
        else:
            remaining_goals.append(goal)

    state["subgoal_list"] = remaining_goals """
    pass
    
def should_continue(state : AgentState) -> str:
    """Determine if we should continue or end the reasoning."""
    last_message = state["messages"][-1]
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        for tool_call in last_message.tool_calls:
            if tool_call["name"] == "plan_from_pddl":
                # Только после вызова plan_from_pddl обновляем подцели
                update_subgoals_from_scene(state)
                break

    if isinstance(last_message, AIMessage):
        try:
            content = last_message.content.strip()
            content = re.sub(r"<think>.*?</think>", "", content, flags=re.DOTALL)
            content = content.strip()
            
            parsed = json.loads(content)
            
            unsolvable = True if "__plan_unsolvable__" in parsed else None
            if unsolvable:
                return "fail"
            elif len(state["subgoal_list"]) == 0: # если все цели выполнены
                return "success"
        except Exception as e:
            print(f"JSON parse error: {e}")
    return "continue"

In [161]:
def tool_executor_node(state: AgentState) -> dict:
    """Custom tool node that injects scene_graph from state into tool calls."""
    messages = state["messages"]
    last_message = messages[-1]

    if not hasattr(last_message, "tool_calls") or not last_message.tool_calls:
        return {"messages": []}

    tool_outputs = []
    for tool_call in last_message.tool_calls:
        tool_name = tool_call["name"]
        args = tool_call["args"]

        if tool_name == "find_object":
            result = find_object.invoke({**args, "graph" :state["scene_graph"]})
        elif tool_name == "get_relations":
            args["object_id"] = int(args["object_id"])
            result = get_relations.invoke({**args, "graph" : state["scene_graph"]})
        elif tool_name == "plan_from_pddl":
            args["pddl_text"] = str(args["pddl_text"])
            result = plan_from_pddl.invoke({**args})
        else:
            result = f"Unknown tool: {tool_name}"

        tool_message = ToolMessage(
            content=str(result),
            name=tool_name,
            tool_call_id=tool_call["id"]
        )
        tool_outputs.append(tool_message)

    return {"messages": tool_outputs}

In [163]:
graph = StateGraph(AgentState)
graph.add_node("agent", my_agent)
graph.add_node("tools", tool_executor_node)

graph.set_entry_point("agent")
graph.add_edge("agent","tools")

graph.add_conditional_edges(
    "tools",
    should_continue,
    {
        "continue" : "agent",
        "fail" : END, # нужно добавить какую-то аналогичную "ошибочную" ноду финала
        "success" : END,
    },
)
app = graph.compile()

In [164]:
def run_model(num_task, max_iterations=10):
    prompt, subgoals = specificate_prompt(num_task, max_iterations)
    _, init_graph = generate_graph_and_task(num_task)
    
    system_prompt = SystemMessage(content=prompt)
    initial_state = AgentState(
        messages=[system_prompt],
        scene_graph=init_graph,
        subgoal_list=subgoals
    )

    state = initial_state
    parsed = None
    for i in range(max_iterations):
        state = app.invoke(state)
        last_message = state['messages'][-1]
        if isinstance(last_message, AIMessage):
            try:
                content = last_message.content.strip()
                content = re.sub(r"<think>.*?</think>", "", content, flags=re.DOTALL).strip()
                parsed = json.loads(content)
                if all(key in parsed for key in [
                    "goals", "relevant_objects", "final_actions"
                ]) or all(key in parsed for key in [
                    "goals", "relevant objects", "final actions"
                ]):
                    print(f"Goal interpretation completed at iteration {i+1}")
                    break
            except Exception as e:
                print(f"Iteration {i+1}: JSON parse error: {e}")
                print(f"Content was: {repr(content[:200])}...")
        else:
            print(f"Iteration {i+1}: Last message not AIMessage")
    else:
        print("Max iterations reached. Returning last result.")

    return  parsed, state['scene_graph'], state['task_description']

In [2]:
from src.action_sequencing.action_sequencing import run_model
run_model(5, 10)

Goal interpretation completed at iteration 1
Subgoal decomposition completed at iteration 1

 AI: 
USING TOOLS: ['find_object']
Scene graph nodes: 280

 AI: 
USING TOOLS: ['find_object']
Scene graph nodes: 280

 AI: 
USING TOOLS: ['find_object']
Scene graph nodes: 280

 AI: 
USING TOOLS: ['get_relations']
Scene graph nodes: 280

 AI: 
USING TOOLS: ['get_relations']
Scene graph nodes: 280

 AI: 
USING TOOLS: ['plan_from_pddl']
Scene graph nodes: 280

 AI: 
USING TOOLS: ['plan_from_pddl']
Scene graph nodes: 280

 AI: 
NO TOOLS CALLED — just thinking...
Scene graph nodes: 280

 AI: <think>
Okay, let me try to figure out why the PDDL is still having unbalanced parentheses. The user mentioned the error is in the problem.pddl. Let me check the code again.

Looking at the problem.pddl section, the goal is written as:

(:goal (and
    (next_to robot.1 bathroom.1)
    (facing robot.1 toilet.37)
    (open toilet.37)
))

Wait, the 'and' is inside the parentheses, but maybe the parentheses aren't 

GraphRecursionError: Recursion limit of 50 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/GRAPH_RECURSION_LIMIT

In [2]:
txt = """ === domain.pddl ===
(define (domain home-robot)
(:requirements :strips :typing)
(:types agent object)
(:predicates
    (next_to ?a - agent ?o - object)
    (facing ?a - agent ?o - object)
    (inside ?o - object ?c - object)
    (clean ?o - object)
    (closed ?o - object)
    (off ?o - object)
)
(:actions
    (:action walk
        :parameters (?a - agent ?o - object)
        :precondition (not (next_to ?a ?o))
        :effect (next_to ?a ?o))
    (:action turnto
        :parameters (?a - agent ?o - object)
        :precondition (next_to ?a ?o)
        :effect (facing ?a ?o))
    (:action open
        :parameters (?o - object)
        :precondition (closed ?o)
        :effect (not (closed ?o)) (off ?o))
    (:action run
        :parameters (?a - agent ?o - object)
        :precondition (next_to ?a ?o)
        :effect (run ?a ?o))
)
)
=== problem.pddl ===
(define (problem robot-task-1)
(:domain home-robot)
(:objects
    robot.1 - agent
    bathroom.1 - object
    toilet.37 - object
)
(:init
    (inside toilet.37 bathroom.1)
    (clean toilet.37)
    (closed toilet.37)
    (off toilet.37)
    (next_to robot.1 bathroom.1)
    (facing robot.1 bathroom.1)
)
(:goal (and
    (next_to robot.1 bathroom.1)
    (facing robot.1 bathroom.1)
    (run robot.1 toilet.37)
))
) """

In [7]:
txt2 = """
=== domain.pddl ===
(define (domain dummy)
  (:requirements :strips)
  (:predicates (a) (b))
  (:action act1
    :precondition (a)
    :effect (b)
  )
  
)
=== problem.pddl ===
(define (problem dummy-prob)
  (:domain dummy)
  (:objects)
  (:init (a))
  (:goal (b))
)
"""

In [14]:
def validate_pddl_output(pddl_text) -> tuple[bool, str]:
    if "=== domain.pddl ===" not in pddl_text:
        return False, "Missing domain.pddl"
    if "=== problem.pddl ===" not in pddl_text:
        return False, "Missing problem.pddl"
    if pddl_text.count("=== domain.pddl ===") > 1:
        return False, "Duplicate domain.pddl"
    if pddl_text.count("=== problem.pddl ===") > 1:
        return False, "Duplicate problem.pddl"
    
    pddl_paths = (Path.cwd() / ".." / ".." / "ff-planner-docker" / "actual_plans").resolve()
    domain_path, problem_path = pddl_paths / "domain.pddl", pddl_paths / "problem.pddl"

    domain_start = pddl_text.find("=== domain.pddl ===") + len("=== domain.pddl ===")
    problem_start = pddl_text.find("=== problem.pddl ===")
    
    if domain_start == -1 or problem_start == -1:
        return False, "Could not locate PDDL blocks"

    # Извлекаем domain (от конца заголовка до начала problem)
    domain_text = pddl_text[domain_start:problem_start].strip()

    # Извлекаем problem (от конца заголовка problem до конца строки)
    problem_end_marker = "=== problem.pddl ==="
    problem_start_idx = pddl_text.find(problem_end_marker) + len(problem_end_marker)
    problem_text = pddl_text[problem_start_idx:].strip()

    # Проверим, что скобки сбалансированы
    if domain_text.count('(') != domain_text.count(')'):
        return False, f"Domain PDDL has unbalanced parentheses. Open: {domain_text.count('(')}, Close: {domain_text.count(')')}"
    if problem_text.count('(') != problem_text.count(')'):
        return False, f"Problem PDDL has unbalanced parentheses. Open: {problem_text.count('(')}, Close: {problem_text.count(')')}"
    
    print(problem_text)
    

    return True, "OK"

In [15]:
validate_pddl_output(txt2)

(define (problem dummy-prob)
  (:domain dummy)
  (:objects)
  (:init (a))
  (:goal (b))
)


(True, 'OK')